# reliability.space
# Reliability Prediction - FORM
# Bearing:     solid lubricant wear model

# by Matthias Schubert, 2021-09-20

For ESA RFP 17225

## General model

Solid lubricant wear modelling is described taking example in a ball bearing. The modelling is applicable to other cases of solid lubricant wear, however, the number of revolution has to be substituted with another measure of sliding distance. 
For solid lubricant reservoir wear (e.g. cage of a ball bearing), the limit state function for the adhesive wear model is formulated as follows:

$g\left( {\bf{X}} \right) = {V_{{\rm{limit}}}} - \Theta  \cdot \sum\limits_{i = 1}^{{n_p}} {{K_{H,i}} \cdot {\alpha _i} \cdot re{v_i}}$

Where $\alpha$ denotes the average work of ball/cage interaction forces per revolution and $rev_i$ the number of revolutions in time interval $i$ . The parameter $\alpha$ will typically be estimated from tests and is, thus dependent on the wear rate $K_H$ , which is used to estimate $\alpha$ from the test results. Therefore, $K_H$ cannot be easily taken out of the sum.

The following calculation is done by using First Order Reliability Analysis (FORM) for one time interval i=1.

In [1]:
# nbi:hide_in
#   IMPORTs
from IPython.display import display 
import ipywidgets as widgets 
from ipywidgets import interact, Layout
import numpy as np
from scipy.stats import norm
from matplotlib import pyplot as plt
from ipywidgets import interact
import nbinteract as nbi

!pip install git+git://github.com/hackl/pyre.git
import pyre
from pyre import *

  Cloning git://github.com/hackl/pyre.git to /tmp/pip-req-build-72o_al49
  Running command git clone -q git://github.com/hackl/pyre.git /tmp/pip-req-build-72o_al49
  Resolved git://github.com/hackl/pyre.git to commit b6b3d40f9b5e57a1c89d88322301f65103fd459d


In [2]:
# nbi:hide_in
def proF(E_Vlim,COV_Vlim,Dist_Vlim,E_KH,COV_KH,Dist_KH,E_alpha,COV_alpha,Dist_alpha,E_MU,COV_MU,Dist_MU,E_nrev):
    
    E_vlimit_val = float(E_Vlim)
    nu_vlimit_val = float(COV_Vlim)
    E_KH_val = float(E_KH)
    nu_KH_val = float(COV_KH)
    E_alpha_val = float(E_alpha)
    nu_alpha_val = float(COV_alpha)
    E_theta_val = float(E_MU)
    nu_theta_val = float(COV_MU)
    rev_val = float(E_nrev)

    rev_min=rev_val*0.5;  #for plotting
    rev_max=rev_val*2;  #for plotting

    #revolutions per hour
    rev_hour=6;
    ######################################################


    # FORM ANALYSIS PyRe
    # Define limit state function
    limit_state = LimitState(lambda X1,X2,X3,X4: X1-X2*X3*X4*rev_val)
    stochastic_model = StochasticModel()
    # Define random variables
    
    if Dist_Vlim=='Normal':
        stochastic_model.addVariable( Normal('X1',E_vlimit_val,E_vlimit_val*nu_vlimit_val) )
    elif Dist_Vlim=='LogNormal':
        stochastic_model.addVariable( Lognormal('X1',E_vlimit_val,E_vlimit_val*nu_vlimit_val) )
    elif Dist_Vlim=='Gumbel':
        stochastic_model.addVariable( Gumbel('X1',E_vlimit_val,E_vlimit_val*nu_vlimit_val) )
    
    if Dist_KH=='Normal':
        stochastic_model.addVariable( Normal('X2',E_KH_val,E_KH_val*nu_KH_val) )
    elif Dist_KH=='LogNormal':
        stochastic_model.addVariable( Lognormal('X2',E_KH_val,E_KH_val*nu_KH_val) )
    elif Dist_KH=='Gumbel':
        stochastic_model.addVariable( Gumbel('X2',E_KH_val,E_KH_val*nu_KH_val) )
    
    if Dist_alpha=='Normal':
        stochastic_model.addVariable( Normal('X3',E_alpha_val,E_alpha_val*nu_alpha_val) )
    elif Dist_alpha=='LogNormal':
        stochastic_model.addVariable( Lognormal('X3',E_alpha_val,E_alpha_val*nu_alpha_val) )
    elif Dist_alpha=='Gumbel':
        stochastic_model.addVariable( Gumbel('X3',E_alpha_val,E_alpha_val*nu_alpha_val) )  
        
    stochastic_model.addVariable( Lognormal('X4',E_theta_val,E_theta_val*nu_theta_val) )
    
    stochastic_model.setCorrelation( CorrelationMatrix([[1.0, 0.0, 0.0, 0.0],
                                                        [0.0, 1.0, 0.5, 0.0],
                                                        [0.0, 0.5, 1.0, 0.0],
                                                        [0.0, 0.0, 0.0, 1.0]]) )
    
    # Set some options (optional)
    options = AnalysisOptions()
    # options.printResults(False)
    
    # Performe FORM analysis
    Analysis = Form(analysis_options=options, stochastic_model=stochastic_model, limit_state=limit_state)

    # Some single results:
    beta = Analysis.getBeta()
    pf = Analysis.getFailure()

    
    #rev_mat=np.linspace(rev_min,rev_max,50);

    #pf_mat=[]
    #for i_rev in range(rev_mat.shape[0]):
    #        E_x2 = (E_KH_val*E_alpha_val + nu_KH_alpha)*rev_mat[i_rev];
    #        nu_x2 = 1/E_x2*rev_mat[i_rev]*np.sqrt(var_KH_alpha);

    #       pf_mat.append( norm.cdf((np.log(E_theta_val) - np.log(E_x1/E_x2) + 0.5*(np.log(nu_x1**2 + 1) -  np.log(nu_x2**2 + 1) - np.log(nu_theta_val**2 + 1))) / np.sqrt(np.log( nu_theta_val**2 + 1 ) + np.log( nu_x1**2 + 1 ) + np.log( nu_x2**2 + 1 ))) )

    #%matplotlib inline
    #plt.plot(np.log10(rev_mat), pf_mat,'b-')
    #plt.plot(np.log10(rev_val), pf,'ro')
    #plt.xlabel('$log_{10}$(number of revolutions)\n solid lubricant wear model')
    #plt.ylabel('Probability of failure $P_f(rev)$')
    #plt.yscale('log')
        
   
    return 'The probability of failure is pf={} at {} revolutions'.format(pf,rev_val)

## Input values for reliability prediction
#### -Limiting value (worn volume), $V_{lim}$ $[m^3]$
#### -Specific wear rate, $K_H$ $[Pa^{-1}=m^2/N]$
#### -Ball-cage interaction, $\alpha$ $[N/m]$
#### -Model uncertainty, $\Theta$ $[-]$
#### -Nominal number of revolutions, $rev$ $[ \# ]$

In [3]:
# nbi:hide_in
interact(proF, E_Vlim='6.5e-8', COV_Vlim='0.2', Dist_Vlim={'LogNormal', 'Normal','Gumbel'}, E_KH='4e-15', COV_KH='0.66', Dist_KH={'LogNormal', 'Normal','Gumbel'}, E_alpha='0.018', COV_alpha='0.2', Dist_alpha={'LogNormal', 'Normal','Gumbel'}, E_MU='1.0', COV_MU='0.2', Dist_MU={'LogNormal'}, E_nrev='245e+6');

interactive(children=(Text(value='6.5e-8', description='E_Vlim'), Text(value='0.2', description='COV_Vlim'), D…

In [4]:
# nbi:hide_in